In [1]:
#Load necessary packages
import tweepy
import folium
import geopy
import os
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#set authentication variables
access_token_eve = os.environ.get('access_token')
access_secret_eve = os.environ.get('access_secret')
consumer_key_eve = os.environ.get('api_key')
consumer_secret_eve = os.environ.get('api_secret')

In [3]:
#Set inital data frame to store tweets for given time frame
#Pull tweet date/time


In [4]:
#load the Twitter API, create def for use
def get_twitter_api():
    # personal details
    consumer_key = consumer_key_eve
    consumer_secret = consumer_secret_eve
    access_token = access_token_eve
    access_token_secret = access_secret_eve
    
    # authentication of consumer key and secret
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    
    # authentication of access token and secret
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

In [17]:
#set search for location of the Twitter user
#incorporate Twitter's own Lon/Lat data into the equation:
#https://developer.twitter.com/en/docs/tutorials/filtering-tweets-by-location

def get_tweets(search):
    api = get_twitter_api()
    location_data = []
    for tweet in tweepy.Cursor(api.search_tweets, q=search).items(15):
        if hasattr(tweet, 'user') and hasattr(tweet.user, 'screen_name') and hasattr(tweet.user, 'location') and hasattr(tweet, 'created_at') and hasattr(tweet, 'id'):
            if tweet.user.location:
                location_data.append((tweet.id, tweet.user.screen_name, tweet.user.location, str(tweet.created_at)))
    return location_data

In [18]:
#pull the tweets and store in data frame 
#data should include -Twitter screen name; location, created date, Tweet ID
search_terms="#Everton"
tweet_batch = get_tweets(search_terms)
print(tweet_batch)

[(1547926057348517889, 'RossAftbl', 'Glasgow/St Albans', '2022-07-15 12:48:25+00:00'), (1547925446796496896, 'fan_banter', 'United Kingdom', '2022-07-15 12:46:00+00:00'), (1547925444560764928, 'matola_olexandr', 'Odessa, Ukraine', '2022-07-15 12:45:59+00:00'), (1547924528625434626, 'StavrateCom', 'Europe', '2022-07-15 12:42:21+00:00'), (1547922996949504003, 'GEEZERBOY1977', 'England, United Kingdom', '2022-07-15 12:36:16+00:00'), (1547919631796473858, 'Fussballeck', 'Deutschland', '2022-07-15 12:22:53+00:00'), (1547913168784986114, 'SimonCBondar', 'Seacombe, England', '2022-07-15 11:57:12+00:00')]


In [19]:
#convert array to NumPy array
tweets=np.array(tweet_batch)
print(tweets)

[['1547926057348517889' 'RossAftbl' 'Glasgow/St Albans'
  '2022-07-15 12:48:25+00:00']
 ['1547925446796496896' 'fan_banter' 'United Kingdom'
  '2022-07-15 12:46:00+00:00']
 ['1547925444560764928' 'matola_olexandr' 'Odessa, Ukraine'
  '2022-07-15 12:45:59+00:00']
 ['1547924528625434626' 'StavrateCom' 'Europe'
  '2022-07-15 12:42:21+00:00']
 ['1547922996949504003' 'GEEZERBOY1977' 'England, United Kingdom'
  '2022-07-15 12:36:16+00:00']
 ['1547919631796473858' 'Fussballeck' 'Deutschland'
  '2022-07-15 12:22:53+00:00']
 ['1547913168784986114' 'SimonCBondar' 'Seacombe, England'
  '2022-07-15 11:57:12+00:00']]


In [20]:
#convert to dataframe
df = pd.DataFrame(tweets, columns=["ID", "Name", "Location", "Date-Time"])
print(df.head())

                    ID             Name                 Location  \
0  1547926057348517889        RossAftbl        Glasgow/St Albans   
1  1547925446796496896       fan_banter           United Kingdom   
2  1547925444560764928  matola_olexandr          Odessa, Ukraine   
3  1547924528625434626      StavrateCom                   Europe   
4  1547922996949504003    GEEZERBOY1977  England, United Kingdom   

                   Date-Time  
0  2022-07-15 12:48:25+00:00  
1  2022-07-15 12:46:00+00:00  
2  2022-07-15 12:45:59+00:00  
3  2022-07-15 12:42:21+00:00  
4  2022-07-15 12:36:16+00:00  


In [35]:
def addLatLon(data):
    #define the geolocator service
    geo_locator = Nominatim(user_agent="EvertonTweeters")
    
    #define the geocoder, with a rate limiter so we don't break Nominatum
    geocode = RateLimiter(geo_locator.geocode, min_delay_seconds=2)
    
    for (Location) in data:
        #geocode the location, then add lat/lon to dataframe; if not a valid location, put "NaN"
        data["Location"] = data["add"].apply(geocode)
        data['Lat'] = data['Location'].apply(lambda x: x.latitude if x else None)
        data["Lon"] = data['Location'].apply(lambda x: x.longitude if x else None)
    
    return data

In [36]:
addLatLon(df)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Deutschland',), **{}).
Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Home\anaconda3\envs\geospatial\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn

,add,location,Lat,Lon,Location
0,Glasgow/St Albans,None,NaN,NaN,None
1,United Kingdom,"(United Kingdom, (54.7023545, -3.2765753))",54.702354,-3.276575,"(United Kingdom, (54.7023545, -3.2765753))"
2,"Odessa, Ukraine","(Одеса, Одеська міська громада, Одеський район...",46.484302,30.732288,"(Одеса, Одеська міська громада, Одеський район..."
3,Europe,"(Ōu, (51.0, 10.0))",51.000000,10.000000,"(Ōu, (51.0, 10.0))"
4,"England, United Kingdom","(England, United Kingdom, (52.5310214, -1.2649...",52.531021,-1.264906,"(England, United Kingdom, (52.5310214, -1.2649..."
5,Deutschland,"(Deutschland, (51.0834196, 10.4234469))",51.083420,10.423447,"(Deutschland, (51.0834196, 10.4234469))"
6,"Seacombe, England","(Seacombe, Birkenhead, Wallasey, Wirral, Liver...",53.408324,-3.022905,"(Seacombe, Birkenhead, Wallasey, Wirral, Liver..."


In [ ]:
def put_markers(map, data):
    for (ID, name, location, time) in data:
        if location:
            try:
                location = geo_locator.geocode(location)
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude], popup=name).add_to(map)

if __name__ == "__main__":
    map = folium.Map(location=[0, 0], zoom_start=2)
    location_data = tweets
    put_markers(map, location_data)
    map.save("index.html")

In [21]:
loc = "Seacombe, England"

geolocator = Nominatim(user_agent="Everton-Tweeters")

location = geolocator.geocode(loc)

print(location.address)
print((location.latitude, location.longitude))


Seacombe, Birkenhead, Wallasey, Wirral, Liverpool City Region, England, CH44 7DP, United Kingdom
(53.4083242, -3.0229049)


In [23]:
locat = ['Glasgow/St Albans', 'United Kingdom', 'Odessa, Ukraine','Europe', 'England, United Kingdom', 'Deutschland','Seacombe, England']

df = pd.DataFrame({'add':locat})

geolocator = Nominatim(user_agent="EvertonTweeters")

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['location'] = df['add'].apply(geocode)
df['Lat'] = df['location'].apply(lambda x: x.latitude if x else None)
df['Lon'] = df['location'].apply(lambda x: x.longitude if x else None)

In [24]:
print(df)

                       add                                           location  \
0        Glasgow/St Albans                                               None   
1           United Kingdom         (United Kingdom, (54.7023545, -3.2765753))   
2          Odessa, Ukraine  (Одеса, Одеська міська громада, Одеський район...   
3                   Europe                                 (Ōu, (51.0, 10.0))   
4  England, United Kingdom  (England, United Kingdom, (52.5310214, -1.2649...   
5              Deutschland            (Deutschland, (51.0834196, 10.4234469))   
6        Seacombe, England  (Seacombe, Birkenhead, Wallasey, Wirral, Liver...   

         Lat        Lon  
0        NaN        NaN  
1  54.702354  -3.276575  
2  46.484302  30.732288  
3  51.000000  10.000000  
4  52.531021  -1.264906  
5  51.083420  10.423447  
6  53.408324  -3.022905  
